# Питон и машинное обучение

## Модуль 1. Инструментарий для машинного обучения

- Библиотеки numpy, pandas и matplotlib
- Загрузка данных из CSV-файлов, таблиц Excel и СУБД
- Разбиение данных:
    - На определяющие и результирующие признаки
    - На обучающую и валидационную выборки 
- Машинное обучение с использованием «сильных алгоритмов»

## Библиотеки numpy, pandas, matplotlib

Стандарт импорта этих библиотек в вашем коде:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('always', category=UserWarning)

### Массивы numpy, особенности

Массивы numpy - основной контейнер данных в задачах машинного обучения.

Особенности этих объектов:
 - обращение к элементам подобно спискам Python (но в одномерном случае)
 - массивы многомерны (чаще всего используются вектора, матрицы и тензоры 3-4-го ранга)
 - в массивах хранятся данные одного типа (как правило, числа)
 - в массивах данные хранятся в одной непрерывной области памяти, это позволяет получить серьезный выигрыш в производительности
 - в numpy встроены собственные функции для агрегатных вычислений (```sum```, ```max```, ```min```, etc), <font color="red">не используйте одноименные функции python c объектами __numpy, pandas, tensorflow, pytorch__, etc</font>.



In [ ]:
a_10 = np.arange(10)
print(a_10)
print(a_10[:5])

In [ ]:
# как визуально отличить массив numpy от списка
print(a_10)
print(list(a_10))

In [ ]:
# векторизация операций
print(a_10 * 10)
print(a_10 + a_10 * 10)

#### Изменение размерности

In [ ]:
a_10.shape

In [ ]:
a_2_5 = a_10.reshape( (2,5) ) # перевод данных в объект другой размерности
a_2_5

In [ ]:
a_5_2 = a_2_5.T # транспонирование
a_5_2

In [ ]:
a_5_2 = a_2_5.transpose(1,0) # транспонирование - 2-й способ
a_5_2

‼️ Все функции изменения размерности возвращают проекцию на ту же область данных, но с другими определениями размерности:

In [ ]:
a_5_2[1,1] = 100500

print(a_5_2)
print(a_2_5)
print(a_10)

In [ ]:
print(a_2_5[:, :2]) # срезы для многомерных массивов numpy
print(a_2_5.flatten()) # "раскатывание в вектор"

#### Векторизация и бродкастинг для многомерного массива

In [ ]:
print(a_2_5 + a_2_5 * 10) # поэлементное сложение, умножение на скаляр
print()
print(a_2_5 * 10  + np.arange(5)) # операция бродкастинг "матрица и строка" 
print()
print(a_2_5 * 10 + np.arange(2).reshape(-1,1)) # операция бродкастинг "матрица и столбец"

#### Фильтрация данных: булевы маски, fancy indexing

In [ ]:
a_20_50 =  a_2_5 * 10
a_20_50

In [ ]:
a_20_50[ a_20_50 >= 70 ] # булева маска

In [ ]:
a_20_50 >= 70

In [ ]:
np.sum(a_20_50 >= 70) # посчитать количество элементов, удовлетворяющих данным условиям

In [ ]:
(a_10 * 10)[ [3,1,2,1] ] # fancy indexing для векторов

In [ ]:
a_20_50[ [0,1,0], [3,2,1]] # fancy indexing для матриц

In [ ]:
a_20_50[ [0,1,0], a_20_50[0] > 10 ] # можно комбинировать, но обращайте внимание на размерность

#### Операция матричного умножения/скалярного произведения .dot

Полезна для любых операций, где требуется поэлементное перемножение элементов с их последующим суммированием. Дает существенный выигрыш в производительности.

In [ ]:
# создаем две матрицы
x = np.array([[1,2],[3,4]])
y = np.array([[5,6],[7,8]])

# и два вектора
v = np.array([9,10])
w = np.array([11, 12])

# Скалярное произведение векторов; оба выражения дают 219
print(v.dot(w))
print(np.dot(v, w))
print()

# Умножение матрицы на вектор, оба выражения возвращают вектор [29 67]
print(np.dot(x, v))
print()

# Умножение матриц, в итоге получаем
# [[19 22]
#  [43 50]]
print(np.dot(x, y))

#### ‼️ Как надо и как не надо пользоваться numpy

Использование numpy неэффективно, если объекты numpy обрабатывать вне самого numpy. Подходите к данным как к векторам и матрицам, обрабатывайте их эффективно!

In [ ]:
# На примере задачи "получить сумму  квадратов натуральных чисел от 0 до 10000"
# чистый python:
n = 10000
%timeit -n200 x = sum(x*x for x in range(n))

# наивный numpy - хуже чистого Python'а
na = np.arange(n)
%timeit -n200 x = sum(na*na)

# векторизованный numpy
%timeit -n200 x = np.dot(na, na)

### Загрузка данных в pandas

Pandas (PANel Data AnalysiS) - библиотека для работы с __табличными__ данными и наш основной контейнер для данных в этом курсе. При работе с ```pandas``` мы будем иметь дело с двумя основными объектами:
- ```pd.DataFrame``` - "каркас для данных", таблица
- ```pd.Series``` - столбец (или, в некоторых случаях, строка) в таблице.

Рассмотрим загрузку данных из различных источников:

1. Загрузка данных из модуля datasets библиотеки sklearn на примере датасета "Ирисы Фишера"

In [ ]:
from sklearn import datasets

iris = datasets.load_iris()
iris

Для решения задач машинного обучения нам нужно сформировать набор определяющих признаков ```X``` и набор результируюих признаков ```y```:

In [ ]:
X = pd.DataFrame(iris.data, columns=iris.feature_names)
X

In [ ]:
y = iris.target
print(type(y))
y

#### Загрузка данных из CSV и Excel-файлов

1. CSV-файлы

In [ ]:
bank = pd.read_csv('data/bank.csv')
bank

#### Более сложный случай, tsv-файл

In [ ]:
traffic = pd.read_csv('data/web_traffic.tsv', 
#                       delimiter='\t',
#                       index_col=0,
#                       header=None,
#                       names=['visitors_per_hour'],
                    )
traffic

#### Как получать данные из базы данных

На примере SQlite, который входит в состав Python. Для других СУБД все аналогично.

In [ ]:
import sqlalchemy as sa

# подготовить connection string
conn_string = "sqlite:///data/girls.db3" 

# создать "движок"
engine = sa.create_engine(conn_string, execution_options={"sqlite_raw_colnames": True})

# получить данные
girls = pd.read_sql_query('SELECT * FROM playboy_model', # SQL-запрос
                          engine, # движок
                          index_col='girl_ID')
girls

#### Как получать данные из таблиц Excel

In [ ]:
!conda install -y openpyxl=3.1 -c conda-forge

In [ ]:
girls_xlsx = pd.read_excel( 'data/girls.xlsx', engine='openpyxl' )
girls_xlsx

### Разбиение набора данных

Как получить определяющий набор данных ```X``` и результирующий набор ```y``` из загруженного датасета?

1. использовать ```iloc``` и индексы, если результирующий признак в начале или в конце:

In [ ]:
X = bank.iloc[:, :-1]
y = bank.iloc[:, -1]
print(y)
X

2. использовать ```loc``` и название признака

In [ ]:
X = bank.loc[:, bank.columns != 'y']
y = bank.loc[:, bank.columns == 'y']
print(y)
X

Получение контрольной и тестовой выборки:

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3, 
                                                    random_state=20240318,
                                                    stratify = y)

In [ ]:
print(X.shape)
print(X_test.shape)
print(X_train.shape)

#### ⁉️ Задание

Загрузите датасет "рукописные цифры" при помощи функции ```load_digits()```, сформируйте ```X``` и ```y``` для этого набора данных, а также сделайте разбиение на тестовую и обучающую выборки в соотношении 20/80. 

Визуализируйте первые несколько цифр.

In [ ]:
# ваш код здесь
from sklearn.datasets import load_digits

digits = load_digits()
X = digits.data
y = digits.target

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3, 
                                                    random_state=20240318,
                                                   stratify = y)

print(X.shape)
print(X_test.shape)
print(X_train.shape)

#### Машинное обучение

Для решения задачи распознавания рукописных цифр будем использовать эффективный алгоритм ```RandomForestClassifier```.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


In [ ]:
forest = RandomForestClassifier(n_estimators=10, max_depth=10, n_jobs=-1) 
forest.fit(X_train,y_train)

test_pred = forest.predict(X_test)

print(f"Accuracy score: {accuracy_score(y_test, test_pred)}")
print()